## 1. Idiomatyczne implementacje metody wytwórczej w Pythonie

Klasyczny wzorzec Metoda Wytwórcza (hierarchia klas z abstrakcyjną metodą) jest **rzadko spotykany** w kodzie Pythona, ponieważ:
  1. Python nie potrzebuje tej struktury - funkcje są bytami pierwszej klasy, mamy słowniki, dekoratory
  2. Idiom Pythona - zamiast hierarchii klas często używamy funkcji/callables
  3. Duck typing eliminuje potrzebę abstrakcyjnych metod

Co zwykle robią Pythoniści zamiast Factory Method?
  - Funkcje wytwórcze (nie metody) (`np.array()`)
  - Alternatywne konstruktory (classmethod) (`dict.fromkeys()`, `pd.DataFrame.from_dict()`)
  - Słownik z callables
  - Proste Fabryki

Klasyczny wzorzec Metoda wytwórcza wymaga hierarchii klas z abstrakcyjną metodą. W Pythonie możemy to znacznie uprościć, wykorzystując na przykład **funkcje** i **słowniki jako rejestry**.

### Problem w klasycznym wzorcu

**Klasyczny wzorzec**:
- Klasa abstrakcyjna `PizzaStore`  
- Abstrakcyjna metoda `create_pizza`
- Osobna klasa dla każdego wariantu (`ItalianPizzaStore`, `AmericanPizzaStore`)

**Pythoniczny idiom**:
- Funkcja wytwórcza lub słownik z callables

### Rozwiązanie 1: Słownik jako rejestr (najprostsze)

Zamiast tworzyć osobne klasy dla każdego wariantu, po prostu używamy słownika.

Jest to możliwe, ponieważ funkcje w pythonie są **bytami pierwszej klasy** (*ang. first-class citizen*). Byt pierwszej klasy, to taki element języka, który może być traktowany jak każda inna wartość -  możemy go przypisywać do zmiennej, przekywać jako parametr i zwracać. Wszystko to jest możliwe, ponieważ funkcje w Pythonie są obiektami. W Pythonie wszystko jest obiektem (poza elementami czysto składniowymi jak: `if`, `return`, `for`, ...), a tym samym wszystko jest bytem pierwszej klasy.

In [22]:
# Zakładamy, że mamy klasy pizzy (uproszczone)
class ItalianCheesePizza:
    name = "Włoska pizza serowa"

class AmericanCheesePizza:
    name = "Amerykańska pizza serowa"

class ItalianPepperoniPizza:
    name = "Włoska pizza pepperoni"

class AmericanPepperoniPizza:
    name = "Amerykańska pizza pepperoni"


# Rejestr statyczny (wartości zahardcodowane)
PIZZA_REGISTRY = {
    ("italian", "cheese"): ItalianCheesePizza,
    ("italian", "pepperoni"): ItalianPepperoniPizza,
    ("american", "cheese"): AmericanCheesePizza,
    ("american", "pepperoni"): AmericanPepperoniPizza,
}


# Prosta funkcja fabryki
def create_pizza(style: str, pizza_type: str):
    """Tworzy pizzę na podstawie stylu i typu"""
    pizza_class = PIZZA_REGISTRY.get((style, pizza_type))
    if pizza_class is None:
        raise ValueError(f"Nieznany typ: {style}, {pizza_type}")
    return pizza_class()


# Użycie
italian_cheese = create_pizza("italian", "cheese")
print(italian_cheese.name)

american_pepperoni = create_pizza("american", "pepperoni")
print(american_pepperoni.name)

Włoska pizza serowa
Amerykańska pizza pepperoni


### Rozwiązanie 2: Dynamiczny rejestr z dekoratorem (bardziej elastyczne)

Jeśli często dodajemy nowe warianty, możemy użyć dekoratora do automatycznej rejestracji:

In [23]:
# Rejestr pizz (dynamiczny rejestr - klasy same się rejestrują w momencie tworzenia)
_pizza_registry = {}

# Dekorator do automatycznej rejestracji
def register_pizza(style, pizza_type):
    def decorator(cls):
        _pizza_registry[(style, pizza_type)] = cls
        return cls
    
    return decorator

# Rejestrujemy pizze używając dekoratora
@register_pizza("italian", "cheese")
class ItalianCheesePizza:
    name = "Włoska pizza serowa"

@register_pizza("american", "cheese")
class AmericanCheesePizza:
    name = "Amerykańska pizza serowa"

@register_pizza("mexican", "spicy")  # Łatwo dodać nową!
class MexicanSpicyPizza:
    name = "Ostra meksykańska pizza"


# Funkcja fabryki
def create_pizza(style: str, pizza_type: str):
    pizza_class = _pizza_registry.get((style, pizza_type))
    if pizza_class is None:
        raise ValueError(f"Nieznany typ: {style}, {pizza_type}")
    return pizza_class()


# Użycie
pizza1 = create_pizza("italian", "cheese")
print(pizza1.name)

pizza2 = create_pizza("mexican", "spicy")
print(pizza2.name)

# Zobacz co jest w rejestrze
print(f"\nDostępne pizze: {list(_pizza_registry.keys())}")

Włoska pizza serowa
Ostra meksykańska pizza

Dostępne pizze: [('italian', 'cheese'), ('american', 'cheese'), ('mexican', 'spicy')]


### Wzorzec vs rozwiązanie idomiatyczne

**Zastosowaliśmy idiomy Pythona**:
1. Funkcje pierwszej klasy (klasy jako wartości w słowniku)
2. Słowniki jako rejestry (mapowanie klucz → callable)
3. Dekoratory do automatycznej rejestracji

**Co zyskaliśmy?**
- **Mniej kodu** - brak hierarchii klas, brak abstrakcyjnych metod
- **Łatwe dodawanie wariantów** - po prostu dodaj wpis do słownika lub użyj dekoratora
- **Czytelne** - od razu widać wszystkie dostępne opcje
- **Elastyczne** - łatwo zmienić logikę wyboru klasy
- **Testowalne** - łatwo mockować rejestr

**Kiedy użyć klasycznego wzorca?**
- Gdy pizzerie potrzebują złożonej logiki inicjalizacji
- Gdy pizzerie dzielą wspólny kod (można użyć dziedziczenia)
- Gdy każda pizzeria ma wiele metod powiązanych ze sobą

**Kiedy użyć słownika/funkcji (idiom)?**
- Prosty wybór: "na podstawie klucza zwróć obiekt" ← **większość przypadków!**
- Gdy często dodajesz nowe warianty
- Gdy chcesz prosty, czytelny kod

**Przykłady wykorzystania**
1. Tworzenie widoków w Django:

```python
# Django - registry view functions
urlpatterns = [
    ('/home/', home_view),
    ('/about/', about_view),
]  # co prawda w tym przypadku rejestr callable nie jest słownikiem tylko listą, ale koncepcyjnie niczego to nie zmienia.

2. Rejestrowanie fixture w pytest
# Pytest - registry fixture factories  
@pytest.fixture
def user():
    return User(name="Test")
```

**W Pythonie rzadko potrzebujesz pełnego wzorca Factory Method - użyj słownika!**